# OPER 610
## Problem 1.13 (Phase 2)
### Brandon Hosley

## Preparation

In [1]:
import numpy as np
from pyomo.environ import * 
from pyomo.opt import SolverFactory

model = ConcreteModel()

## Definitions

Sets:
- Products: $P = \{1,2,3\}$
- Machines: $M = \{1,2,3,4\}$
	
Parameters:
- $c_{pm}$: Production Cost (a matrix drawn from the first table)
- $t_{pm}$: Hours to produce units of product (a matrix drawn from the second table)
- $u_p$   : Units of product $p$ required
- $h_m$   : Hours machines $m$ available

In [2]:
# Labels for products and machines
model.P = Set(initialize=range(3))
model.M = Set(initialize=range(4))

# VARS
model.x = Var(model.P, model.M, domain=NonNegativeReals)

# Production constants
model.c = np.array([[4,4,5,7],
                    [6,7,5,6],
                    [12,10,8,11]])

model.t = np.array([[0.3,0.25,0.2,0.2],
                    [0.2,0.3,0.2,0.25],
                    [0.8,0.6,0.6,0.5]])  

# Scenario input/outputs
model.u = np.array([3000,6000,4000])
model.h = np.array([1500,1200,1500,2000])

3 Set Declarations
    M : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {0, 1, 2, 3}
    P : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    P*M :   12 : {(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3)}

1 Var Declarations
    x : Size=12, Index=x_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :  None :  None : False :  True : NonNegativeReals
        (0, 1) :     0 :  None :  None : False :  True : NonNegativeReals
        (0, 2) :     0 :  None :  None : False :  True : NonNegativeReals
        (0, 3) :     0 :  None :  None : False :  True : NonNegativeReals
        (1, 0) :     0 :  None :  None : False :  Tru

## Objective Function

$\operatorname{min}\quad \sum_{p\in P} \sum_{m\in M} c_{pm}x_{pm}$

In [3]:
model.OBJ = Objective(expr=sum(model.x[p,m]*model.c[p,m] for p in model.P for m in model.M), sense=minimize)

## Constraints

$\sum_{m\in M} x_{pm} = u_p  \qquad \text{for } p\in P $
$\sum_{p\in P} t_{pm}x_{pm}  \leq h_m  \quad \text{for } m\in M $
        
*Non-negativity constrained above*

In [4]:
model.constraints = ConstraintList()

# Meet production goals
for p in model.P:
    model.constraints.add(sum(model.x[p,m] for m in model.M) == model.u[p])
    
# Machine operating time limits
for m in model.M:
    model.constraints.add(sum(model.x[p,m]*model.t[p,m] for p in model.P) <= model.h[m])

## Solve

In [5]:
solver = SolverFactory('glpk')
sol = solver.solve(model, report_timing=True, tee=True)

        0.01 seconds required to write file
        0.05 seconds required for presolve
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmpg53s8cx3.glpk.raw
 --wglp /var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmp_013xr4p.glpk.glp
 --cpxlp /var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmpwpa6cxks.pyomo.lp
Reading problem data from '/var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmpwpa6cxks.pyomo.lp'...
8 rows, 13 columns, 25 non-zeros
81 lines were read
Writing problem data to '/var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmp_013xr4p.glpk.glp'...
69 lines were written
GLPK Simplex Optimizer, v4.65
8 rows, 13 columns, 25 non-zeros
Preprocessing...
7 rows, 12 columns, 24 non-zeros
Scaling...
 A: min|aij| =  2.000e-01  max|aij| =  1.000e+00  ratio =  5.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 7
      0: obj =   1.050000000e+05 inf 

In [6]:
model.pprint()

4 Set Declarations
    M : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {0, 1, 2, 3}
    P : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    P*M :   12 : {(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3)}

1 Var Declarations
    x : Size=12, Index=x_index
        Key    : Lower : Value  : Upper : Fixed : Stale : Domain
        (0, 0) :     0 : 3000.0 :  None : False : False : NonNegativeReals
        (0, 1) :     0 :    0.0 :  None : False : False : NonNegativeReals
        (0, 2) :     0 :    

In [7]:
print("The solver terminated with the following decision variable values:")              
for x in model.x:
    if(value(model.x[x])>0):
       print("  ", x, format(value(model.x[x]),".2f"))

The solver terminated with the following decision variable values:
   (0, 0) 3000.00
   (1, 2) 1500.00
   (1, 3) 4500.00
   (2, 1) 2000.00
   (2, 2) 2000.00
